# Vector store

In [14]:
#!pip uninstall -y -q gensim
!pip install -q gensim

In [15]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models import KeyedVectors
import heapq
from collections import defaultdict, namedtuple
import numpy as np # we mneed numpy 1.26.4
from numpy import float32 as FLOAT_TYPE



In [16]:
# 1. Parameters
VECTOR_SIZE = 1000
WINDOW      = 5
MIN_COUNT   = 5
WORKERS     = 4
MAX_WORDS_IN_BATCH = 1000

### CustomLineSentence

In [17]:
import itertools
from gensim import utils

class CustomLineSentence:
    def __init__(self, source):
        self.source = source

    def __iter__(self):
        """Iterate through the lines in the source."""
        with utils.open(self.source, 'rb') as fin:
            for line in itertools.islice(fin, None):
                line = utils.to_unicode(line).split()
                i = 0
                while i < len(line):
                    yield line[i: i + MAX_WORDS_IN_BATCH]
                    i += MAX_WORDS_IN_BATCH


In [18]:
# 2. Load the text8 corpus as a stream of sentences
sentences = CustomLineSentence('data/Royal_data.txt')

print("Number of sentences:", len(list(sentences)))

Number of sentences: 11


In [19]:
print("Type of 'sentences':", type(sentences))
print("What we got from 'sentences':")
for i, s in enumerate(itertools.islice(sentences, 3)):
    print(f"Sentence {i+1}:", s)

    print("Length of sentence:", len(s))
    


Type of 'sentences': <class '__main__.CustomLineSentence'>
What we got from 'sentences':
Sentence 1: ['The', 'future', 'king', 'is', 'the', 'prince']
Length of sentence: 6
Sentence 2: ['Daughter', 'is', 'the', 'princess']
Length of sentence: 4
Sentence 3: ['Son', 'is', 'the', 'prince']
Length of sentence: 4


In [20]:
# limit to the first 100 sentences for downstream processing
sentences = list(sentences)[:100]
print("Number of sentences in subset:", len(sentences))

Number of sentences in subset: 11


# Custom word2vec class
by Creating bigrams

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter

class BigramModel(nn.Module):
    def __init__(self, input_size, embed_size, output_size):
        super(BigramModel, self).__init__()
        self.fc1 = nn.Linear(input_size, embed_size)  # Embedding layer: projects one-hot input to dense vector
        self.fc2 = nn.Linear(embed_size, output_size) # Output layer: projects embedding to vocabulary size logits
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.softmax(x, dim=1)  # Apply softmax to convert logits to probabilities over vocabulary

class CustomBigramWord2Vec:
    def __init__(self, documents=None, vector_size=100, window=5, min_count=5, epochs=500, workers=4, batch_size=256):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.batch_size = batch_size
        self.workers = workers
        self.wv = {}
        self.index_to_key = []
        
        if documents is not None:
            self.build_vocab(documents)
            self.train_model()
    
    def build_vocab(self, documents):
        """Build vocabulary and create bigrams from documents"""
        # Flatten the list of documents into a list of words
        all_words = [word for doc in documents for word in doc]
        
        # Count word occurrences and filter by min_count
        word_counts = Counter(all_words)

        print("word_counts: ", word_counts)

        filtered_words = [word for word, count in word_counts.items() if count >= self.min_count]
        
        # Create word-to-index and index-to-word mappings
        self.index_to_key = sorted(filtered_words)

        print("index_to_key: ", self.index_to_key)

        self.word_to_idx = {word: i for i, word in enumerate(self.index_to_key)}
        self.vocab_size = len(self.index_to_key)

        print(f"word_to_idx: {self.word_to_idx}")
        
        print(f"Vocabulary size after filtering: {self.vocab_size}")
        
        print("documents: ", documents)

        # Create bigram pairs for training
        self.bigrams = []
        for doc in documents:
            filtered_doc = [word for word in doc if word in self.word_to_idx]
            print("filtered_doc: ", filtered_doc)
            for i in range(len(filtered_doc)):
                # Consider words within the window
                for j in range(max(0, i - self.window), min(len(filtered_doc), i + self.window + 1)):
                    if i != j:  # Skip the word itself
                        self.bigrams.append([filtered_doc[i], filtered_doc[j]])
        
        print(f"Created {len(self.bigrams)} bigram pairs for training")

        print("bigram samples: ", self.bigrams)
        
        # Create one-hot encoding dictionary
        self.onehot_dict = {}
        for i, word in enumerate(self.index_to_key):
            one_hot = torch.zeros(self.vocab_size)
            one_hot[i] = 1
            self.onehot_dict[word] = one_hot
            
        # Prepare training data
        self.X = []
        self.Y = []
        
        for bi in self.bigrams:
            if bi[0] in self.onehot_dict and bi[1] in self.onehot_dict:
                self.X.append(self.onehot_dict[bi[0]])
                self.Y.append(self.onehot_dict[bi[1]])
                
        self.X = torch.stack(self.X)
        self.Y_indices = torch.tensor([self.word_to_idx[bi[1]] for bi in self.bigrams 
                                      if bi[0] in self.onehot_dict and bi[1] in self.onehot_dict], 
                                     dtype=torch.long)

    def train_model(self):
        """Train the bigram model"""
        self.model = BigramModel(self.vocab_size, self.vector_size, self.vocab_size)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters())
        
        for epoch in range(self.epochs):
            permutation = torch.randperm(self.X.size()[0])
            epoch_loss = 0.0
            for i in range(0, self.X.size()[0], self.batch_size):
                indices = permutation[i:i+self.batch_size]
                batch_x, batch_y = self.X[indices], self.Y_indices[indices]
                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            if (epoch+1) % 100 == 0:
                print(f"Epoch {epoch+1}/{self.epochs}, Loss: {epoch_loss:.4f}")
        
        # Extract word embeddings from the trained model
        weights = self.model.fc1.weight.data.T.numpy()  # shape: (vocab_size, vector_size)
        for i, word in enumerate(self.index_to_key):
            self.wv[word] = weights[i]

    def save(self, path):
        """Save model to disk"""
        data = {
            'wv': self.wv,
            'index_to_key': self.index_to_key,
            'vector_size': self.vector_size,
            'window': self.window,
            'min_count': self.min_count,
        }
        torch.save(data, path)
    
    @classmethod
    def load(cls, path):
        """Load model from disk"""
        data = torch.load(path)
        model = cls()
        model.wv = data['wv']
        model.index_to_key = data['index_to_key']
        model.vector_size = data['vector_size']
        model.window = data['window']
        model.min_count = data['min_count']
        return model
    

## Custom Doc2Vec Class
Use Gensim's Doc2Vec and TaggedDocument to build and use document vectors.

In [70]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
import pickle

class CustomDoc2Vec:
    def __init__(self, documents, vector_size=100, window=5, min_count=5, epochs=5, workers=4, negative=5):
        self.vector_size = vector_size
        self.documents = documents
        self.doc_vectors = np.zeros((len(documents), vector_size), dtype=FLOAT_TYPE)
        
        # Train Word2Vec on the documents
        self.word_model = CustomBigramWord2Vec(
            documents=documents,
            vector_size=vector_size,
            window=window,
            min_count=min_count,
            workers=workers,
            epochs=epochs
        )
        
        # Generate document vectors by averaging word vectors
        for i, doc in enumerate(documents):
            valid_words = [word for word in doc if word in self.word_model.wv]
            if valid_words:
                self.doc_vectors[i] = np.mean([self.word_model.wv[word] for word in valid_words], axis=0)
    
    def infer_vector(self, document):
        """
        Infer a vector for a new document by averaging its word vectors
        """
        valid_words = [word for word in document if word in self.word_model.wv]
        if not valid_words:
            return np.zeros(self.vector_size, dtype=FLOAT_TYPE)
        return np.mean([self.word_model.wv[word] for word in valid_words], axis=0)
    
    def save(self, path):
        """Save model to file"""
        with open(path, 'wb') as f:
            pickle.dump({
                'word_model': self.word_model,
                'doc_vectors': self.doc_vectors,
                'vector_size': self.vector_size,
                'documents': self.documents
            }, f)
    
    @classmethod
    def load(cls, path):
        """Load model from file"""
        with open(path, 'rb') as f:
            data = pickle.load(f)
        
        obj = cls.__new__(cls)
        obj.word_model = data['word_model']
        obj.doc_vectors = data['doc_vectors']
        obj.vector_size = data['vector_size']
        obj.documents = data['documents']
        return obj

## Document Vector Store
Build document embeddings and search functionality

In [71]:
import numpy as np
from numpy.linalg import norm
from nltk.corpus import stopwords
import nltk


class DocumentVectorStore:
    def __init__(self, documents, embedding_model=None):
        """documents: list of token lists or raw text strings"""
        self.raw_documents = documents
        self.filter_stopwords = True


        # get stopwords rom nltk
        if self.filter_stopwords:
            try:
                self.stopwords = set(stopwords.words('english'))
            except:
                print("downloading stopwords...")
                nltk.download('stopwords')
                self.stopwords = set(stopwords.words('english'))


        # tokenize 
        self.tokenized_docs = [doc.split() if isinstance(doc, str) else doc for doc in documents]

        # If stopword filtering is enabled, process each document
        if self.filter_stopwords:
            processed = []  # initialize list to hold processed token lists
            for doc in self.raw_documents: 
                # split raw text into tokens if it's a string, otherwise use the list as is
                tokens = doc.split() if isinstance(doc, str) else doc
                # filter out tokens that are in the stopword set (case-insensitive)
                processed.append([tok for tok in tokens if tok.lower() not in self.stopwords])
                self.tokenized_docs = processed  # assign the filtered token lists
        
        self.model = embedding_model
        # build document embeddings
        self.build_doc_embeddings()

    def build_doc_embeddings(self):
        """Compute document embeddings by averaging word vectors"""
        embeddings = []
        print("model", self.model)
        for tokens in self.tokenized_docs:
            vecs = [self.model.wv[word] for word in tokens if word in self.model.wv]
            if vecs:
                doc_vec = np.mean(vecs, axis=0)
            else:
                doc_vec = np.zeros(self.model.vector_size)
            embeddings.append(doc_vec)
        self.doc_embeddings = np.vstack(embeddings)
    
    def search(self, query, topn=5):
        """Return topn most similar documents for a query (tokens or string)"""
        if isinstance(query, str):
            tokens = query.split()
        else:
            tokens = query
        
        # Filter stopwords if enabled
        if self.filter_stopwords:
            tokens = [word for word in tokens if word.lower() not in self.stopwords]

        # Define vecs before using it
        vecs = [self.model.wv[word] for word in tokens if word in self.model.wv]
        
        if vecs:
            q_vec = np.mean(vecs, axis=0)
        else:
            q_vec = np.zeros(self.model.vector_size)
            
        # cosine similarity
        sims = self.doc_embeddings.dot(q_vec) / (norm(self.doc_embeddings, axis=1) * norm(q_vec) + 1e-9)
        idx = np.argsort(-sims)[:topn]
        
        return [(i, float(sims[i])) for i in idx]


In [72]:
print("Number of documents:", len(sentences))

# Train our CustomDoc2Vec model
doc2vec_model = CustomDoc2Vec(
    documents=sentences,
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    epochs=1000,
    workers=WORKERS,
    negative=5
)

# Save and reload the Doc2Vec model
doc2vec_model.save('./models/custom_doc2vec.model')
loaded_doc2vec = CustomDoc2Vec.load('./models/custom_doc2vec.model')

# Build a DocumentVectorStore using the trained Doc2Vec embeddings
doc_store = DocumentVectorStore(sentences, embedding_model=loaded_doc2vec.word_model)

# testing search
query_doc = sentences[0]
results = doc_store.search(query_doc, topn=5)
print("Top 5 similar documents (index, similarity):", results)


Number of documents: 11
index_to_key:  ['a', 'be', 'is']
word_to_idx: {'a': 0, 'be': 1, 'is': 2}
Vocabulary size after filtering: 3
documents:  [['The', 'future', 'king', 'is', 'the', 'prince'], ['Daughter', 'is', 'the', 'princess'], ['Son', 'is', 'the', 'prince'], ['Only', 'a', 'man', 'can', 'be', 'a', 'king'], ['Only', 'a', 'woman', 'can', 'be', 'a', 'queen'], ['The', 'princess', 'will', 'be', 'a', 'queen'], ['The', 'prince', 'is', 'a', 'strong', 'man'], ['The', 'princess', 'is', 'a', 'beautiful', 'woman'], ['Prince', 'is', 'only', 'a', 'boy', 'now'], ['Prince', 'will', 'be', 'king'], ['A', 'boy', 'will', 'be', 'a', 'man']]
filtered_doc:  ['is']
filtered_doc:  ['is']
filtered_doc:  ['is']
filtered_doc:  ['a', 'be', 'a']
filtered_doc:  ['a', 'be', 'a']
filtered_doc:  ['be', 'a']
filtered_doc:  ['is', 'a']
filtered_doc:  ['is', 'a']
filtered_doc:  ['is', 'a']
filtered_doc:  ['be']
filtered_doc:  ['be', 'a']
Created 22 bigram pairs for training
bigram samples:  [['a', 'be'], ['a', 'a'],

In [34]:
import textwrap

In [35]:
query_doc = sentences[1]
# join all words into one long string
text = " ".join(query_doc)

# wrap at 80 chars per line
print(textwrap.fill(text, width=80))

Daughter is the princess


In [36]:
searchterm = """experience however in practice many autistic people have difficulty with working
in groups which impairs them even in the most technical of situations autistic
adults temple grandin one of the more successful adults with autism photograph
courtesy joshua nathaniel pritikin and william lawrence jarrold some autistic"""

In [41]:
sample_search_term = "king"

In [46]:
results = doc_store.search(sample_search_term, topn=20)
print(f"Top 5 similar documents for {sample_search_term}, (index, similarity):", results,)

Top 5 similar documents for king, (index, similarity): [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0)]


In [47]:
# join all words into one long string
query_doc = sentences[1]
text = " ".join(query_doc)

# wrap at 80 chars per line
print(textwrap.fill(text, width=80))

Daughter is the princess


# We want to use PCA, WHY.

PCA (Principal Component Analysis) is used to reduce the dimensionality of high-dimensional data, such as embeddings, while preserving as much of the original information (variance) as possible.

We are going to give it `n_components=3` to reduce the embeddings to 3 dimensions.

In my test case, we just test the first 100 words.

Also, what I can see when I look at the data is that we have a lot of words that are stopwords. We are going to want to remove them.

In [48]:
from sklearn.decomposition import PCA

numOfWords = 10

weights = doc2vec_model.model.wv.vectors

print(weights.shape)  # (number of words, vector size)
import matplotlib.pyplot as plt

# Reduce vectors to 3D
pca = PCA(n_components=3)
weights_3d = pca.fit_transform(weights)

# Plot the first 1000 words in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(weights_3d[:numOfWords, 0], weights_3d[:numOfWords, 1], weights_3d[:numOfWords, 2])

# Annotate points with words
words = list(doc2vec_model.model.wv.index_to_key)
for i in range(numOfWords):
    ax.text(weights_3d[i, 0], weights_3d[i, 1], weights_3d[i, 2], words[i], size=8)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.title('Word Embeddings in 3D')
plt.show()


AttributeError: 'CustomDoc2Vec' object has no attribute 'model'

In [ ]:
# Use PCA to reduce document embeddings to 3D and plot them

# Get document embeddings from doc_store
doc_embeddings = doc_store.doc_embeddings

print(doc_embeddings.shape)  # (number of documents, vector size)

# Reduce to 3D
pca = PCA(n_components=3)
doc_emb_3d = pca.fit_transform(doc_embeddings)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(doc_emb_3d[:100, 0], doc_emb_3d[:100, 1], doc_emb_3d[:100, 2])

# Annotate points with document indices
for i in range(100):
    ax.text(doc_emb_3d[i, 0], doc_emb_3d[i, 1], doc_emb_3d[i, 2], str(i), size=8)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.title('Document Embeddings in 3D')
plt.show()

# Evaluation of Vector Store Solution

Let's compare our custom vector store solution with other standard approaches to evaluate its performance.

In [ ]:
!pip install tabulate